In [3]:
!pip install requests
!pip install tzdata

In [4]:
import sys
import json
import os.path
import requests
import time
from datetime import datetime,timedelta
from time import mktime
#from zoneinfo import ZoneInfo

# for enabling and disabling the subs to view shit which is outdated and not valid anymore
THEORETICAL_VIEWS = False

# from colorama import init
# from colorama import Fore, Style
# init()

HOLO_POI_URL = "https://holoapi.poi.cat/api/v4/channels_report?ids={holo}&metrics=youtube_channel_subscriber,youtube_channel_view&startAt={startDate}&endAt={endDate}"
NASFAQ_URL = "https://nasfaq.biz/api/getStats"

JSON_FILENAME = "holostats.json"


# subs are possibly worth about 10k view for 1k subs
# this doesn't apply anymore but I'm too lazy to scrub it out
SUB_TO_VIEW_VALUE = 10000

# list the holos by their generation according to nasfaq since
# the lists returned by neither server does so
# girls you dont want can just be removed from the list
HOLOS = ["hololive", "sora", "roboco", "miko", "suisei", "azki", 
         "mel", "fubuki", "matsuri", "aki", "haato", 
         "aqua", "shion", "ayame", "choco", "subaru", 
         "mio", "okayu", "korone", 
         "pekora", "rushia", "flare", "noel", "marine", 
         "kanata", "coco", "watame", "towa", "himemoriluna", 
         "lamy", "nene", "botan", "polka", 
         "risu", "moona", "iofi", 
         "calliope", "kiara", "inanis", "gura", "amelia", "irys", 
         "sana", "fauna", "kronii", "mumei", "baelz",
         "ollie", "melfissa", "reine", 
         "ui", "nana", "pochimaru", "ayamy", "nabi"]

# need to ensure we dont get fucked by dumb time fuckery which includes dst
#TIMEZONE = ZoneInfo("Australia/Brisbane")
#values for when adjustments occurs
LOCAL_RESET_HOUR = 13
#nasfaq
LOCAL_ADJUSTMENT_MINUTE = 5
#this is for holopoi
LOCAL_RESET_MINUTE = 0
# 12 hours
REDOWNLOAD_STAT_TIME = 43200
# testHolo = "kanata"

# with open("data_file.json", "w") as write_file:
#     json.dump(data, write_file)
currentTime = int(time.time() * 1e3) 
threeDaysMilliSeconds = 259200000

threeDaysAgoTime = int((currentTime - threeDaysMilliSeconds))
fourDaysMilliSeconds = 345600000

fourDaysAgoTime = int((currentTime - fourDaysMilliSeconds))

#response = requests.get(testHoloURL)
statResponseJson = None
if( not os.path.isfile(JSON_FILENAME)):
    print("Creating {filename} and getting getStat".format(filename=JSON_FILENAME))
    res = requests.get(NASFAQ_URL)
    res = res.json()
    statResponseJson = res
    curTime = mktime(datetime.now().timetuple())
    jsonData = {}
    jsonData["lastGrab"] = curTime
    jsonData["stats"] = res
    with open(JSON_FILENAME, "w", encoding="utf-8") as statJSON:
        json.dump(jsonData, statJSON)
else:
    res = None
    with open(JSON_FILENAME, "r+", encoding="utf-8") as statJSON:
        res = json.load(statJSON)
    curTime = mktime(datetime.now().timetuple())
    if (curTime - res["lastGrab"]) > REDOWNLOAD_STAT_TIME:
        print("Updating getStat")
        res = requests.get(NASFAQ_URL)
        res = res.json()
        statResponseJson = res
        jsonData = {}
        jsonData["lastGrab"] = curTime
        jsonData["stats"] = res
        with open(JSON_FILENAME, "w", encoding="utf-8") as statJSON:
            json.dump(jsonData, statJSON)
    else:
        print("Using saved stats")
        statResponseJson = res["stats"]

coinHistory = json.loads(statResponseJson['coinHistory'])

currentDate = datetime.now()
dateFormat = "{month}/{day}/{year}"

currentDateStamp = dateFormat.format(month = currentDate.strftime("%m"), day = currentDate.strftime("%d"), year = currentDate.strftime("%Y"))

holoList = statResponseJson['stats']
poiStatCount ={}

yesterdayAdjustmentTime = coinHistory[-1]['timestamp']
# for tick in reversed(range(len(statResponseJson['coinInfo']['data']['aki']['history']))):
#     pass
#     curTimestamp = statResponseJson['coinInfo']['data']['aki']['history'][tick]['timestamp']
#     curDate = datetime.fromtimestamp(curTimestamp / 1e3)
#     curHour = curDate.hour
#     curMinute = curDate.minute
#     if (curHour == LOCAL_RESET_HOUR and LOCAL_ADJUSTMENT_MINUTE == curDate.minute and LOCAL_ADJUSTMENT_MINUTE == (curDate.minute - 1) and LOCAL_ADJUSTMENT_MINUTE == (curDate.minute + 1) ):
#         yesterdayAdjustmentTime = curTimestamp

yesterdayAdjustmentHistory = {}

# for holo in statResponseJson['coinInfo']['data']:
#     pass
#     history = statResponseJson['coinInfo']['data'][holo]['history']
#     for tick in reversed(range(len(history))):
#         if(history[tick]['timestamp'] == yesterdayAdjustmentTime):
#             pass
#             yesterdayAdjustmentHistory[holo] = history[tick]
for tick in reversed(range(len(coinHistory))):
    pass
    history = coinHistory
    if(history[tick]['timestamp'] == yesterdayAdjustmentTime):
        pass
        yesterdayAdjustmentHistory = history[tick]['data']
todayCoinHistory = {}
for holo in coinHistory[-1]['data']:
    pass
    holoCoin = coinHistory[-1]['data'][holo]
    todayCoinHistory[holo] = holoCoin

# todayCoinHistory = statResponseJson['todayPrices'][-1]["coinInfo"]['data']


Creating holostats.json and getting getStat


In [5]:
yesterday = currentDate - timedelta(days=1)
twoDaysAgo = currentDate - timedelta(days=2)
threeDaysAgo = currentDate - timedelta(days=3)
#print(yesterday)
#print(twoDaysAgo)

for holo in HOLOS:
    if (holo == 'civia'):
        continue
    if(holo == 'kronii'):
      holoPoiUrl = HOLO_POI_URL.format(holo='ouro', startDate=fourDaysAgoTime, endDate=currentTime)
      poiStatReponse = requests.get(holoPoiUrl)
    elif(holo == 'fauna'):
      holoPoiUrl = HOLO_POI_URL.format(holo='ceres', startDate=fourDaysAgoTime, endDate=currentTime)
      poiStatReponse = requests.get(holoPoiUrl)
    elif(holo == 'baelz'):
      holoPoiUrl = HOLO_POI_URL.format(holo='hakos', startDate=fourDaysAgoTime, endDate=currentTime)
      poiStatReponse = requests.get(holoPoiUrl)
    elif(holo == 'luna'):
      holoPoiUrl = HOLO_POI_URL.format(holo='himemoriluna', startDate=fourDaysAgoTime, endDate=currentTime)
      poiStatReponse = requests.get(holoPoiUrl)
    else:
      holoPoiUrl = HOLO_POI_URL.format(holo=holo, startDate=fourDaysAgoTime, endDate=currentTime)
      poiStatReponse = requests.get(holoPoiUrl)

    #print(poiStatReponse.json())
    nasDailySubCount = holoList[holo]['dailySubscriberCount']['data'][-1]
    nasDailyViewCount = holoList[holo]['dailyViewCount']['data'][-1]

    holoPoiStatCount = {}

    yesterday = currentDate - timedelta(days=1)
    twoDaysAgo = currentDate - timedelta(days=2)
    threeDaysAgo = currentDate - timedelta(days=3)
    

    for report in poiStatReponse.json()['reports']:
        pass
        # print(report)
        # print('\n\n\n\n')
        rows = report['rows']
        poiTodayCount = {"time":rows[-1][0], "value":rows[-1][1]}
        poiYesterdayCount = {}
        poiTwoDaysCount ={}
        poiThreeDaysCount ={}
        poiDailyCount = 0

        statCounts = {}
        statCounts['today'] = poiTodayCount

        for row in reversed(range(len(rows)-1)):
            pass
            time = rows[row][0]
            unixTime = time/1e3
            # print(unixTime)
            date = datetime.fromtimestamp(unixTime)
            day = date.day
            hour = date.hour
            minutes = date.minute
            # print(date)
            # print(day == yesterday.day)
            # print(hour == LOCAL_RESET_HOUR)
            # print(minutes == LOCAL_RESET_MINUTE)

            
            if (day == yesterday.day and hour == LOCAL_RESET_HOUR and minutes == LOCAL_RESET_MINUTE):
                poiYesterdayCount = {"time":rows[row][0], "value":rows[row][1]}
                statCounts['yesterday'] = poiYesterdayCount

            elif(day == twoDaysAgo.day and hour == LOCAL_RESET_HOUR and minutes == LOCAL_RESET_MINUTE):
                poiTwoDaysCount = {"time":rows[row][0], "value":rows[row][1]}
                statCounts['twoDays'] = poiTwoDaysCount

            elif(day == threeDaysAgo.day and hour == LOCAL_RESET_HOUR and minutes == LOCAL_RESET_MINUTE):
                poiThreeDaysCount = {"time":rows[row][0], "value":rows[row][1]}
                statCounts['threeDays'] = poiThreeDaysCount

        if (report['kind'] == "youtube_channel_subscriber"):
            pass
            holoPoiStatCount['subs'] = statCounts
        elif (report['kind'] == 'youtube_channel_view'):
            pass
            holoPoiStatCount['views'] = statCounts
    poiStatCount[holo] = holoPoiStatCount
    # break



2021-10-14 06:52:54.311777
2021-10-13 06:52:54.311777


In [6]:
holoBuy = []
holoSell = []
holoBuySmall = []
holoSellSmall = []
holoSubTick = []
holoSubDown = []
holoNotUpdated = []

with open('holostats.txt', 'w') as statFile:

    def PrintStats(holo):
        holoPoiStatCount = poiStatCount[holo]

        todayViews = holoPoiStatCount['views']['today']['value']
        yesterdayViews = holoPoiStatCount['views']['yesterday']['value']
        twoDaysViews = holoPoiStatCount['views']['twoDays']['value']
        threeDaysViews = holoPoiStatCount['views']['threeDays']['value']

        todaySubs = holoPoiStatCount['subs']['today']['value']
        yesterdaySubs = holoPoiStatCount['subs']['yesterday']['value']
        twoDaysSubs = holoPoiStatCount['subs']['twoDays']['value']
        threeDaysSubs = holoPoiStatCount['subs']['threeDays']['value']

        dailyViewDifference = todayViews - yesterdayViews + 1
        yesterdayViewDifference = yesterdayViews - twoDaysViews + 1
        twoDaysViewDifference = twoDaysViews - threeDaysViews + 1

        dailySubDifference = todaySubs - yesterdaySubs
        yesterdaySubDifference = yesterdaySubs - twoDaysSubs
        twoDaysSubDifference = twoDaysSubs - threeDaysSubs

        nasWeeklyViewCount = holoList[holo]["weeklyViewCount"]["data"][-1]
        averageViews = int(nasWeeklyViewCount) / 7


        theoreticalSubViews = 0
        theoreticalYesterdaySubViews = 0
        if(dailySubDifference / 1000 != 0):
            theoreticalSubViews = (dailySubDifference / 1000) * SUB_TO_VIEW_VALUE
        if(yesterdaySubDifference / 1000 != 0):
            theoreticalYesterdaySubViews = (yesterdaySubDifference / 1000) * SUB_TO_VIEW_VALUE


        # print(todayViews)
        # print(yesterdayViews)
        # print(dailyViewDifference)

        # print (Fore.BLUE)

        ##print(holo)

        # print(Style.RESET_ALL)

        holoPoiStatCount=poiStatCount[holo]

        ##print("Today views - {views}".format(views=todayViews))
        ##print("Yesterday views - {views}".format(views=yesterdayViews))
        ##print("Daily difference - {difference}".format(difference=(dailyViewDifference)))
        ##print("Yesterday daily difference - {difference}".format(difference=(yesterdayViewDifference)))
        ##print("Day difference - {difference}".format(difference=(dailyViewDifference - yesterdayViewDifference)))
        if(dailyViewDifference != 1):
            if(yesterdayViewDifference != 0):
                difference=((float(dailyViewDifference / yesterdayViewDifference) -1) * 100)
                #Rename himemoriluna to luna in results screen
                if(holo=='himemoriluna'):
                  holo='luna'

                
                if (difference > 15 and dailyViewDifference > 100000):
                  holoBuy.append({"1": holo, "2diff": int(difference), "3": dailyViewDifference, "4":yesterdayViewDifference, "5":twoDaysViewDifference, "6ave": int(averageViews)})
                elif (difference < -15 and dailyViewDifference - yesterdayViewDifference < -100000):
                  holoSell.append({"1": holo, "2diff": int(difference), "3": dailyViewDifference, "4":yesterdayViewDifference, "5":twoDaysViewDifference, "6ave": int(averageViews)})
                elif (difference > 1):
                  holoBuySmall.append({"1": holo, "2diff": int(difference), "3": dailyViewDifference, "4":yesterdayViewDifference, "5":twoDaysViewDifference, "6ave": int(averageViews)})
                elif (difference < -1):
                  holoSellSmall.append({"1": holo, "2diff": int(difference), "3": dailyViewDifference, "4":yesterdayViewDifference, "5":twoDaysViewDifference, "6ave": int(averageViews)})

        else:
          holoNotUpdated.append(holo)
                ##print("Daily % difference - {difference}%".format(difference=((float(dailyViewDifference / yesterdayViewDifference) -1) * 100)))
        ##print("Average weekly views: {average}".format(average=averageViews))

        ##print("\nToday subs - {views}".format(views=todaySubs))
        ##print("Yesterday subs - {views}".format(views=yesterdaySubs))
        ##print("Daily difference - {difference}".format(difference=(dailySubDifference)))
        if (dailySubDifference > 5000):
          holoSubTick.append({"holo": holo, "subDiff": dailySubDifference})
        ##print("Yesterday daily difference - {difference}".format(difference=(yesterdaySubDifference)))

        ##print ("\n---------------------------------")


    for holo in poiStatCount:
        PrintStats(holo)

In [15]:
#Buy table: lists Holos with a daily difference of over 100k views with a view growth of at least 15% from previous day
print("Buy Table\n-----------------------------------------------------------------------------")
for i in sorted(holoBuy, key=lambda k: k['2diff'], reverse=True):
  if(i["4"] > i["5"]):
    overbuy = "\033[1;32;1m"
  else:
    overbuy = "\033[1;31;1m"
  if(i["3"]/i["5"] < 1):
    overbuy = overbuy + "*"
  print(overbuy + i["1"] + "      \t Diff: " + str(i["2diff"]) + "%   \tTodays: " + str(i["3"]) + "     \tYest: " + str(i["4"]) + "       \t2D: " + str(i["5"]) + "         \tAve: " + str(i["6ave"]))

Buy Table
-----------------------------------------------------------------------------
nene      	 Diff: 543%   	Todays: 180695     	Yest: 28096       	2D: 145537         	Ave: 48510
gura      	 Diff: 318%   	Todays: 962603     	Yest: 230147       	2D: 253724         	Ave: 579443
rushia      	 Diff: 276%   	Todays: 340230     	Yest: 90466       	2D: 310310         	Ave: 241989
pekora      	 Diff: 240%   	Todays: 676641     	Yest: 198479       	2D: 440093         	Ave: 431792
*kiara      	 Diff: 220%   	Todays: 173596     	Yest: 54110       	2D: 259080         	Ave: 163864
*fauna      	 Diff: 219%   	Todays: 185810     	Yest: 58125       	2D: 191671         	Ave: 108161
*irys      	 Diff: 184%   	Todays: 147749     	Yest: 51948       	2D: 172434         	Ave: 171517
*inanis      	 Diff: 172%   	Todays: 157852     	Yest: 57914       	2D: 411228         	Ave: 173688
*subaru      	 Diff: 159%   	Todays: 314373     	Yest: 121117       	2D: 337769         	Ave: 343847
*luna      	 Diff: 50%

In [16]:
#Small Buy table: Holos with at least 1% view growth from previous day, but may not have significant view numbers (shitcoins)
print("Small Buy Table\n-----------------------------------------------------------------------------")
for i in sorted(holoBuySmall, key=lambda k: k['2diff'], reverse=True):
  if(i["4"] > i["5"]):
    overbuy = "\033[1;32;1m"
  else:
    overbuy = "\033[1;31;1m"
  if(i["3"]/i["5"] < 1):
    overbuy = overbuy + "*"
  print(overbuy + i["1"] + "      \t Diff: " + str(i["2diff"]) + "%   \tTodays: " + str(i["3"]) + "     \tYest: " + str(i["4"]) + "       \t2D: " + str(i["5"]) + "         \tAve: " + str(i["6ave"]))

Small Buy Table
-----------------------------------------------------------------------------
pochimaru      	 Diff: 721%   	Todays: 21337     	Yest: 2596       	2D: 2602         	Ave: 10472
iofi      	 Diff: 413%   	Todays: 32841     	Yest: 6394       	2D: 8803         	Ave: 17113
nana      	 Diff: 136%   	Todays: 45457     	Yest: 19250       	2D: 19892         	Ave: 19535
reine      	 Diff: 102%   	Todays: 94213     	Yest: 46535       	2D: 89361         	Ave: 64456
choco      	 Diff: 94%   	Todays: 90315     	Yest: 46419       	2D: 16466         	Ave: 33786
moona      	 Diff: 36%   	Todays: 91423     	Yest: 66919       	2D: 76409         	Ave: 83970
nabi      	 Diff: 25%   	Todays: 5142     	Yest: 4108       	2D: 3833         	Ave: 4868
azki      	 Diff: 15%   	Todays: 20573     	Yest: 17873       	2D: 18728         	Ave: 19322
mel      	 Diff: 11%   	Todays: 10337     	Yest: 9243       	2D: 9836         	Ave: 10385
risu      	 Diff: 9%   	Todays: 11707     	Yest: 10708       	2D: 11

In [17]:
#Sell table: List of Holos whose daily view numbers dropped by more than 100k views and have a drop of at least 15% view growth
print("Sell Table\n-----------------------------------------------------------------------------")
for i in sorted(holoSell, key=lambda k: k['2diff']):
  if(i["4"] > i["5"]):
    overbuy = "\033[1;32;1m"
  else:
    overbuy = "\033[1;31;1m"
  if(i["3"]/i["5"] > 1):
    overbuy = overbuy + "*"
  print(overbuy + i["1"] + "      \t Diff: " + str(i["2diff"]) + "%   \tTodays: " + str(i["3"]) + "     \tYest: " + str(i["4"]) + "       \t2D: " + str(i["5"]) + "         \tAve: " + str(i["6ave"]))

Sell Table
-----------------------------------------------------------------------------
roboco      	 Diff: -83%   	Todays: 52235     	Yest: 325746       	2D: 91980         	Ave: 125029
flare      	 Diff: -72%   	Todays: 80417     	Yest: 291497       	2D: 274171         	Ave: 179595
amelia      	 Diff: -67%   	Todays: 73599     	Yest: 224947       	2D: 275003         	Ave: 176153
lamy      	 Diff: -65%   	Todays: 60443     	Yest: 173631       	2D: 158738         	Ave: 178715
*mio      	 Diff: -57%   	Todays: 102439     	Yest: 241587       	2D: 48912         	Ave: 147303
polka      	 Diff: -49%   	Todays: 128531     	Yest: 255419       	2D: 311761         	Ave: 268624
fubuki      	 Diff: -47%   	Todays: 121899     	Yest: 230805       	2D: 124314         	Ave: 208471
*ollie      	 Diff: -46%   	Todays: 204141     	Yest: 378900       	2D: 168823         	Ave: 133267
*kanata      	 Diff: -35%   	Todays: 197850     	Yest: 305217       	2D: 187564         	Ave: 248634
suisei      	 Diff: -2

In [18]:
#Small sell table: list of Holos with at least 1% drop in daily views and a day to day change of less than 100k
print("Small Sell Table\n-----------------------------------------------------------------------------")
for i in sorted(holoSellSmall, key=lambda k: k['2diff']):
  if(i["4"] > i["5"]):
    overbuy = "\033[1;32;1m"
  else:
    overbuy = "\033[1;31;1m"
  if(i["3"]/(i["5"]+1) > 1):
    overbuy = overbuy + "*"
  print(overbuy + i["1"] + "      \t Diff: " + str(i["2diff"]) + "%   \tTodays: " + str(i["3"]) + "     \tYest: " + str(i["4"]) + "       \t2D: " + str(i["5"]) + "         \tAve: " + str(i["6ave"]))

Small Sell Table
-----------------------------------------------------------------------------
*sana      	 Diff: -83%   	Todays: 15282     	Yest: 91969       	2D: 11282         	Ave: 38830
sora      	 Diff: -52%   	Todays: 49937     	Yest: 104562       	2D: 87121         	Ave: 106108
ayamy      	 Diff: -49%   	Todays: 979     	Yest: 1923       	2D: 4090         	Ave: 3214
noel      	 Diff: -36%   	Todays: 111001     	Yest: 174839       	2D: 193168         	Ave: 189959
*melfissa      	 Diff: -30%   	Todays: 14695     	Yest: 21184       	2D: 4958         	Ave: 17485
baelz      	 Diff: -28%   	Todays: 31033     	Yest: 43324       	2D: 191527         	Ave: 178677
korone      	 Diff: -20%   	Todays: 94518     	Yest: 119171       	2D: 315376         	Ave: 222917
aki      	 Diff: -17%   	Todays: 20124     	Yest: 24334       	2D: 46241         	Ave: 62503
towa      	 Diff: -17%   	Todays: 163672     	Yest: 197494       	2D: 174809         	Ave: 175894
mumei      	 Diff: -16%   	Todays: 30013 

In [11]:
#Holos that haven't updated yet for the day
holoNotUpdated

[]

In [12]:
#Holos with a subtick of > 5000
holoSubTick

[{'holo': 'rushia', 'subDiff': 10000}, {'holo': 'gura', 'subDiff': 10000}]

In [13]:
#Holos both with a significant subtick and on the buy table (BUYBUYBUY)
for item in holoSubTick:
  for ele in holoBuy:
    if (item["holo"] == ele["1"]):
      print(item["holo"])

rushia
gura
